### necessary imports

In [1]:
#imaging libraries
import matplotlib.pyplot as plt
import numpy as np
import PIL

#neural network imports
import keras
from keras.layers import Input, Dense, Flatten
from keras.models import model_from_json

In [27]:
#other imports throughout this notebook
# import matplotlib.image as mpimg
# from glob import glob
# import seaborn as sns

### re-read in data

In [4]:
#object inheritence is not really necessary
data_dir = "C://Users//leode//Blood-Cancer_Data//Official//"

trainDf = glob(data_dir + "train/*")
valDf = glob(data_dir + "validation/*")
testDf = glob(data_dir + "test/*")

tr_size = len(trainDf)
val_size = len(valDf)
test_size = len(testDf)

#dimensions of each image
nrow, ncol, rgb = mpimg.imread(trainDf[0]).shape

In [5]:
#initialization of train, val, test (omitting train)
# X_train = np.ones((tr_size, nrow, ncol, rgb))
# y_train = np.ones((tr_size,))

X_val = np.ones((val_size, nrow, ncol, rgb))
y_val = np.ones((val_size,))

X_test = np.ones((test_size, nrow, ncol, rgb))
y_test = np.ones((test_size,))

In [10]:
#put in values for X_val and X_test

print("Starting with the validation data...")
for i, p in enumerate(valDf):
    X_val[i] = mpimg.imread(valDf[i])/255.
    y_val[i] = int(p[-5])
    if i%800==0: print(i)
        
print("Ending with the testing data")
for i, p in enumerate(testDf):
    X_test[i] = mpimg.imread(testDf[i])/255.
    y_test[i] = int(p[-5])
    if i%300==0: print(i)


Starting with the validation data...
0
800
1600
Ending with the testing data
0
300
600
900


In [6]:
labels = ["Healthy", "Acute Lymphoblastic Leukemia",
          "Acute myeloid leukemia", "Multiple Myeloma", "Chronic Myeloid Leukemia"]

### read in each model... later retrain the neural net

In [7]:
# load json and create model
nnParent = "nnModels//"
json_file = open(nnParent + 'model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(nnParent + "model2.h5")
print("Loaded model from disk")

Loaded model from disk


In [8]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#score = loaded_model.evaluate(X, Y, verbose=0)

In [9]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 268, 358, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 119, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 117, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 29, 39, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 37, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 9, 12, 64)        0

##### make sure to read in the ANN file as well

#### evaluating on test data... probably translate this to a different notebook

In [86]:
#make this generalizable...
def testMetrics(X_test, y_test, model=loaded_model, nsamples='all'):
    if nsamples == 'all':
        numSamples = X_test.shape[0]
    else: numSamples = nsamples
    #percentage accuracy
    numCorrect = 0
    #confidence deviation
    confDiff = 0
    for i, img in enumerate(X_test):
        if i==numSamples: break
        img = np.expand_dims(img, axis=0)
        predScores = model.predict(img, verbose=0)
        pred = np.argmax(predScores)
        if (pred == y_test[i]):
            #this prediction is correct...
            numCorrect += 1
            confDiff += 1.000000 - predScores[0][pred]
        else:
            #do nothing, so weighted won't change
            confDiff += 1
            pass
    
    return (float(numCorrect/numSamples), float(confDiff/numSamples) )
    

In [88]:
testMetrics(patch_data, y_val, nsamples=1130)

(0.1929203539823009, 0.9219327173665561)

## the intepretive algorithms

### method 1: patching

In [44]:
#best to change the output visual to a heatmap or something
def patchImage(image, clf, window_size=(50,50)):
    pred_score = clf.predict(np.expand_dims(image, axis=0)) #should output five distinct float values
    idx = np.argmax(pred_score)
    patch_scores = np.full((image.shape[0], image.shape[1]), np.max(pred_score))
    PATCH = np.zeros((window_size[0], window_size[1], 3))
    row_end, col_end = window_size
    while row_end < image.shape[0]:
        while col_end < image.shape[1]:
            #the patch is applied to coordinates with row_end and col_end
            patched_img = image.copy() #to avoid corrupting the original image
            r1, r2, c1, c2 = (row_end-window_size[0], row_end, col_end-window_size[1], col_end)
            patched_img[r1:r2,c1:c2] = PATCH
#             plt.imshow(patched_img)
#             plt.show()
            #print("\n\n-----------------")
            p_score = np.max(clf.predict( np.expand_dims(patched_img, axis=0), verbose=0 ))
            patch_scores[r1:r2, c1:c2] = np.full(window_size, p_score)
            col_end += window_size[0]
            #print(p_score)
        col_end = window_size[0]
        row_end += window_size[1]
        
#     plt.imshow(image)
#     #but also display the significant values
#     plt.show()
#     plt.imshow(np.patch_scores)
    #visualization code is BELOW:
#     hmap = sns.heatmap(patch_scores)
#     hmap.set_title("Confidence deviations after patching")
#     hmap.set_xticks([])
#     hmap.set_yticks([])
#     sns.color_palette("light:#5A9", as_cmap=True)
    return patch_scores

In [30]:
###how each patched image will be put in...
#patch_score_0 = patchImage(X_test[0], loaded_model, )

In [35]:
#very short initialization of patches
patch_data = np.zeros((val_size, 270, 360, 3))

In [52]:
for i, img in enumerate(X_val):
    patch_score_i = patchImage(img, loaded_model)
    patch_data[i, :, :, 0] = patch_score_i
    patch_data[i, :, :, 1] = patch_score_i
    patch_data[i, :, :, 2] = patch_score_i
    print("KEEP GOING at i =", i, "\n\n",)

1/1 [==============================] - 0s 48ms/step
KEEP GOING at i = 0 


1/1 [==============================] - 0s 47ms/step
KEEP GOING at i = 1 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 2 


1/1 [==============================] - 0s 52ms/step
KEEP GOING at i = 3 


1/1 [==============================] - 0s 67ms/step
KEEP GOING at i = 4 


1/1 [==============================] - 0s 46ms/step
KEEP GOING at i = 5 


1/1 [==============================] - 0s 48ms/step
KEEP GOING at i = 6 


1/1 [==============================] - 0s 74ms/step
KEEP GOING at i = 7 


1/1 [==============================] - 0s 55ms/step
KEEP GOING at i = 8 


1/1 [==============================] - 0s 46ms/step
KEEP GOING at i = 9 


1/1 [==============================] - 0s 53ms/step
KEEP GOING at i = 10 


1/1 [==============================] - 0s 60ms/step
KEEP GOING at i = 11 


1/1 [==============================] - 0s 46ms/step
KEEP GOING at i = 12 


1/1 [=================

1/1 [==============================] - 0s 51ms/step
KEEP GOING at i = 108 


1/1 [==============================] - 0s 45ms/step
KEEP GOING at i = 109 


1/1 [==============================] - 0s 56ms/step
KEEP GOING at i = 110 


1/1 [==============================] - 0s 47ms/step
KEEP GOING at i = 111 


1/1 [==============================] - 0s 170ms/step
KEEP GOING at i = 112 


1/1 [==============================] - 0s 49ms/step
KEEP GOING at i = 113 


1/1 [==============================] - 0s 64ms/step
KEEP GOING at i = 114 


1/1 [==============================] - 0s 111ms/step
KEEP GOING at i = 115 


1/1 [==============================] - 0s 55ms/step
KEEP GOING at i = 116 


1/1 [==============================] - 0s 49ms/step
KEEP GOING at i = 117 


1/1 [==============================] - 0s 54ms/step
KEEP GOING at i = 118 


1/1 [==============================] - 0s 46ms/step
KEEP GOING at i = 119 


1/1 [==============================] - 0s 89ms/step
KEEP GOING at i = 120 

1/1 [==============================] - 0s 52ms/step
KEEP GOING at i = 214 


1/1 [==============================] - 0s 54ms/step
KEEP GOING at i = 215 


1/1 [==============================] - 0s 55ms/step
KEEP GOING at i = 216 


1/1 [==============================] - 0s 69ms/step
KEEP GOING at i = 217 


1/1 [==============================] - 0s 55ms/step
KEEP GOING at i = 218 


1/1 [==============================] - 0s 64ms/step
KEEP GOING at i = 219 


1/1 [==============================] - 0s 54ms/step
KEEP GOING at i = 220 


1/1 [==============================] - 0s 56ms/step
KEEP GOING at i = 221 


1/1 [==============================] - 0s 74ms/step
KEEP GOING at i = 222 


1/1 [==============================] - 0s 57ms/step
KEEP GOING at i = 223 


1/1 [==============================] - 0s 64ms/step
KEEP GOING at i = 224 


1/1 [==============================] - 0s 52ms/step
KEEP GOING at i = 225 


1/1 [==============================] - 0s 56ms/step
KEEP GOING at i = 226 



1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 320 


1/1 [==============================] - 0s 81ms/step
KEEP GOING at i = 321 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 322 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 323 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 324 


1/1 [==============================] - 0s 48ms/step
KEEP GOING at i = 325 


1/1 [==============================] - 0s 51ms/step
KEEP GOING at i = 326 


1/1 [==============================] - 0s 48ms/step
KEEP GOING at i = 327 


1/1 [==============================] - 0s 55ms/step
KEEP GOING at i = 328 


1/1 [==============================] - 0s 60ms/step
KEEP GOING at i = 329 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 330 


1/1 [==============================] - 0s 49ms/step
KEEP GOING at i = 331 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 332 



1/1 [==============================] - 0s 91ms/step
KEEP GOING at i = 426 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 427 


1/1 [==============================] - 0s 60ms/step
KEEP GOING at i = 428 


1/1 [==============================] - 0s 48ms/step
KEEP GOING at i = 429 


1/1 [==============================] - 0s 57ms/step
KEEP GOING at i = 430 


1/1 [==============================] - 0s 58ms/step
KEEP GOING at i = 431 


1/1 [==============================] - 0s 88ms/step
KEEP GOING at i = 432 


1/1 [==============================] - 1s 546ms/step
KEEP GOING at i = 433 


1/1 [==============================] - 0s 423ms/step
KEEP GOING at i = 434 


1/1 [==============================] - 0s 54ms/step
KEEP GOING at i = 435 


1/1 [==============================] - 0s 53ms/step
KEEP GOING at i = 436 


1/1 [==============================] - 0s 57ms/step
KEEP GOING at i = 437 


1/1 [==============================] - 0s 50ms/step
KEEP GOING at i = 438 

...

1/1 [==============================] - 0s 143ms/step
KEEP GOING at i = 744 


1/1 [==============================] - 0s 70ms/step
KEEP GOING at i = 745 


1/1 [==============================] - 0s 90ms/step
KEEP GOING at i = 746 


1/1 [==============================] - 0s 98ms/step
KEEP GOING at i = 747 


1/1 [==============================] - 0s 70ms/step
KEEP GOING at i = 748 


1/1 [==============================] - 0s 100ms/step
KEEP GOING at i = 749 


1/1 [==============================] - 0s 70ms/step
KEEP GOING at i = 750 


1/1 [==============================] - 0s 77ms/step
KEEP GOING at i = 751 


1/1 [==============================] - 0s 70ms/step
KEEP GOING at i = 752 


1/1 [==============================] - 0s 165ms/step
KEEP GOING at i = 753 


1/1 [==============================] - 0s 90ms/step
KEEP GOING at i = 754 


1/1 [==============================] - 0s 88ms/step
KEEP GOING at i = 755 


1/1 [==============================] - 0s 90ms/step
KEEP GOING at i = 756

1/1 [==============================] - 0s 75ms/step
KEEP GOING at i = 850 


1/1 [==============================] - 0s 135ms/step
KEEP GOING at i = 851 


1/1 [==============================] - 0s 207ms/step
KEEP GOING at i = 852 


1/1 [==============================] - 0s 111ms/step
KEEP GOING at i = 853 


1/1 [==============================] - 0s 189ms/step
KEEP GOING at i = 854 


1/1 [==============================] - 0s 204ms/step
KEEP GOING at i = 855 


1/1 [==============================] - 0s 158ms/step
KEEP GOING at i = 856 


1/1 [==============================] - 0s 77ms/step
KEEP GOING at i = 857 


1/1 [==============================] - 0s 83ms/step
KEEP GOING at i = 858 


1/1 [==============================] - 0s 96ms/step
KEEP GOING at i = 859 


1/1 [==============================] - 0s 244ms/step
KEEP GOING at i = 860 


1/1 [==============================] - 0s 99ms/step
KEEP GOING at i = 861 


1/1 [==============================] - 0s 131ms/step
KEEP GOING at i 

1/1 [==============================] - 0s 60ms/step
KEEP GOING at i = 955 


1/1 [==============================] - 0s 189ms/step
KEEP GOING at i = 956 


1/1 [==============================] - 0s 187ms/step
KEEP GOING at i = 957 


1/1 [==============================] - 0s 225ms/step
KEEP GOING at i = 958 


1/1 [==============================] - 0s 212ms/step
KEEP GOING at i = 959 


1/1 [==============================] - 0s 140ms/step
KEEP GOING at i = 960 


1/1 [==============================] - 0s 184ms/step
KEEP GOING at i = 961 


1/1 [==============================] - 0s 230ms/step
KEEP GOING at i = 962 


1/1 [==============================] - 0s 180ms/step
KEEP GOING at i = 963 


1/1 [==============================] - 0s 222ms/step
KEEP GOING at i = 964 


1/1 [==============================] - 0s 188ms/step
KEEP GOING at i = 965 


1/1 [==============================] - 0s 384ms/step
KEEP GOING at i = 966 


1/1 [==============================] - 0s 237ms/step
KEEP GOING a

1/1 [==============================] - 0s 84ms/step
KEEP GOING at i = 1060 


1/1 [==============================] - 0s 85ms/step
KEEP GOING at i = 1061 


1/1 [==============================] - 0s 60ms/step
KEEP GOING at i = 1062 


1/1 [==============================] - 0s 210ms/step
KEEP GOING at i = 1063 


1/1 [==============================] - 0s 92ms/step
KEEP GOING at i = 1064 


1/1 [==============================] - 0s 123ms/step
KEEP GOING at i = 1065 


1/1 [==============================] - 0s 80ms/step
KEEP GOING at i = 1066 


1/1 [==============================] - 0s 451ms/step
KEEP GOING at i = 1067 


1/1 [==============================] - 0s 165ms/step
KEEP GOING at i = 1068 


1/1 [==============================] - 0s 82ms/step
KEEP GOING at i = 1069 


1/1 [==============================] - 0s 85ms/step
KEEP GOING at i = 1070 


1/1 [==============================] - 0s 80ms/step
KEEP GOING at i = 1071 


1/1 [==============================] - 0s 109ms/step
KEEP GO

KeyboardInterrupt: 

In [ ]:
#template code for patch_score visualization... add more please!
hmap = sns.heatmap(patch_scores)
hmap.set_title("Confidence deviations after patching")
hmap.set_xticks([])
hmap.set_yticks([])
sns.color_palette("light:#5A9", as_cmap=True)

In [ ]:
#make below information into a markdown...
"""to test confidence, simply compare the following:
    *let Y = exp_test.predict(X_data_down[i]) WITH
    *let Y' = exp_test.predict(pimg[i])....
   Criteria can be based on avg difference of chosen values, in which if
   np.argmax(Y) != np.argmax(Y'), then push that to a queue Wr of wrong indexes
"""

In [63]:
#performance metrics code for Patching
###for windows_size = (50, 50): (0.1929203539823009, 0.9219327173665561)
###for window_size = (30, 30): ()
###for window_size = (10, 10)

1/1 [==============================] - 0s 190ms/step


array([[0.4031959 , 0.4373529 , 0.2065741 , 0.36279407, 0.40659228]],
      dtype=float32)

### method 2: sensitivity analysis

In [ ]:
#start working on this code!!!!!


### method 3: reverse propagation

In [92]:
firstLay = 7
ignoreLays = {8}

In [98]:
#this class assumes the user knows where the flattened and dense layers reside
class RevProp:
    
    def __init__(self, model, startLay, ignoreLays):
        self.layers = model.layers
        self.endLay = startLay
        self.ignoreLays = ignoreLays
        
    """Things to not forget (checklist)
        * consider biases, which differ between answers
        * the call for weights is self.layers[currLay].weights[0][:, answer]
        * experiment on criteria for top Q in currLay (not simply highest values)
        * ...
    """
    
    #there are only five possible permutations for value returns
    #find if there is a way to get different weights/biases when putting an image
    def revprop(self, answer, currLay, Q):
        if currLay==self.endLay:
            #not sure what to return for base case
            return 0
            
        if currLay in self.ignoreLays: 
            self.revprop(answer, currLay-1, Q)
            return 0 #BUG FIXED
        
        highestVals = {}
        print(answer, "--->", currLay)
        weights = self.layers[currLay].weights[0][:, answer]
        for i, w in enumerate(weights.numpy()):
            highestVals[w] = i
            
        #sort by weights
        valOrder = sorted(highestVals, reverse=True)
        for A in range(Q):
            vkey = valOrder[A]
            #print(vkey, "---", highestVals[vkey], "---", currLay)
            self.revprop(highestVals[vkey], currLay-1, int(Q/2))
            
            
        
        
#already globaly defined in the notebook
revprop = RevProp(loaded_model, firstLay, ignoreLays)

In [99]:
#proper call of the algorithm
#revprop.revprop(answer=<>, currLay=<10/len_layers>, Q=<8 or so>)
revprop.revprop(2, 10, 8)

2 ---> 10
4 ---> 9
39 ---> 9
2 ---> 9
24 ---> 9
26 ---> 9
49 ---> 9
25 ---> 9
44 ---> 9
